![image](https://raw.githubusercontent.com/Trouble404/kaggle-Job-Salary-Prediction/master/readme_pic/describtion.PNG)

# Load Dataset 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [14]:
First_Valid = pd.read_csv('dataset\Valid_rev1.csv')
First_Valid.head()

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SourceName
0,13656201,"Lead Technical Architect, C Banking",Lead Technical Architect required for a Tier *...,London,London,NaN,permanent,Scope AT Limited,IT Jobs,jobserve.com
1,14663195,RECRUITMENT CONSULTANT INDUSTRIAL / COMMERCIA...,THIS IS A LIVE VACANCY NOT A GENERIC ADVERTISE...,"LEEDS, West Yorkshire",Leeds,NaN,permanent,Code Blue Recruitment,HR & Recruitment Jobs,cv-library.co.uk
2,16530664,Mechanical / Chemical / Process Engineer Cool...,Mechanical / Chemical / Process Engineer Cool...,"Hampshire, South East",Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs,cv-library.co.uk
3,19047458,Trainee Mortgage Advisor West Midlands,"Are you a successful, results driven person? A...",West Midlands,West Midlands,NaN,permanent,Brite Recruitment,Accounting & Finance Jobs,cv-library.co.uk
4,20881907,Mortgage Services Consultant East Midlands,"Are you a successful, results driven person? D...",East Midlands,East Midlands,NaN,permanent,Brite Recruitment,Accounting & Finance Jobs,cv-library.co.uk


** What's the use of valid data? Difference between trian data?**

In [7]:
First_Train = pd.read_csv('dataset\Train_rev1.csv')
First_Train.head()

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk
1,12612830,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,"Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 35000/annum 25-35K,30000,cv-library.co.uk
2,12612844,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,"Hampshire, South East, South East",Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 40000/annum 20-40K,30000,cv-library.co.uk
3,12613049,Engineering Systems Analyst / Mathematical Mod...,Engineering Systems Analyst / Mathematical Mod...,"Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 30000/annum 25K-30K negotiable,27500,cv-library.co.uk
4,12613647,"Pioneer, Miser Engineering Systems Analyst","Pioneer, Miser Engineering Systems Analyst Do...","Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk


**Train dataset contains 244768 rows × 12 columns**

* Id - A unique identitier for each job ad.
* Title - A freetext field supplied to us by the job advertiser as the Title of the job ad.
* LocationRaw - The freetext location as provided by the job advertiser.
* LocationNormalized - Adzuna's normalised location from within our own location tree, interpreted by us based on the raw location. This normaliser is not **perfect**.
* ContractType - full_time or part_time, interpreted by Adzuna from description or a specific additional field we received from the advertiser.
* ContractTime - permanent or contract, interpreted by Adzuna from description or a specific additional field we received from the advertiser.
* Company - the name of the employer as supplied to us by the job advertiser.
* Category - which of 30 standard job categories this ad fits into, inferred in a very messy way based on the source the ad came from. There is a lot of **noise** and **error** in this field.
* SalaryRaw - the freetext salary field we received in the job advert from the advertiser.
* SalaryNormalised - the annualised salary interpreted by Adzuna from the raw salary.  Note that this is always a single value based on the midpoint of any range found in the raw salary.  This is the value we are trying to predict.
* SourceName - the name of the website or advertiser from whom we received the job advert. 

In [9]:
First_Test = pd.read_csv('dataset\Test_rev1.csv')
First_Test.head()

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SourceName
0,11888454,Business Development Manager,The Company: Our client is a national training...,"Tyne Wear, North East",Newcastle Upon Tyne,NaN,permanent,Asset Appointments,Teaching Jobs,cv-library.co.uk
1,11988350,Internal Account Manager,The Company: Founded in **** our client is a U...,"Tyne and Wear, North East",Newcastle Upon Tyne,NaN,permanent,Asset Appointments,Consultancy Jobs,cv-library.co.uk
2,12612558,Engineering Systems Analysts,Engineering Systems Analysts Surrey ****K Loca...,"Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,cv-library.co.uk
3,12613014,CIS Systems Engineering Consultant,CIS Systems Engineering Consultant Bristol So...,"Bristol, South West, South West",Bristol,NaN,permanent,Gregory Martin International,Engineering Jobs,cv-library.co.uk
4,22454872,CNC Miller / Programmer Fanac,"CNC Miller / Programmer Fanac Fleet, Hampshire...","Fleet, Hampshire",Fleet,NaN,permanent,Gregory Martin International,Manufacturing Jobs,cv-library.co.uk


**Test dataset contains 122463 rows × 10 columns**

It will be used to predict the **SalaryRaw** and **SalaryNormalised**

In [13]:
Location_Tree = pd.read_csv('dataset\Location_Tree.csv')
Location_Tree.head()

,UK~London~East London~Mile End
0,UK~London~East London~Shadwell
1,UK~London~East London~Spitalfields
2,UK~London~East London~Stepney
3,UK~London~East London~Wapping
4,UK~London~East London~Whitechapel


This is a supplemental data set that describes the hierarchical relationship between the different Normalised Locations shown in the job data.  It it is likely that there are meaningful relationships between the salaries of jobs in a similar geographical area, for example average salaries in London and the South East are higher than in the rest of the UK.

# Feature engineering

In [10]:
# Train dataset
First_Train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244768 entries, 0 to 244767
Data columns (total 12 columns):
Id                    244768 non-null int64
Title                 244767 non-null object
FullDescription       244768 non-null object
LocationRaw           244768 non-null object
LocationNormalized    244768 non-null object
ContractType          65442 non-null object
ContractTime          180863 non-null object
Company               212338 non-null object
Category              244768 non-null object
SalaryRaw             244768 non-null object
SalaryNormalized      244768 non-null int64
SourceName            244767 non-null object
dtypes: int64(2), object(10)
memory usage: 22.4+ MB


In [11]:
# Test dataset
First_Test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122463 entries, 0 to 122462
Data columns (total 10 columns):
Id                    122463 non-null int64
Title                 122463 non-null object
FullDescription       122463 non-null object
LocationRaw           122463 non-null object
LocationNormalized    122463 non-null object
ContractType          33013 non-null object
ContractTime          90702 non-null object
Company               106202 non-null object
Category              122463 non-null object
SourceName            122463 non-null object
dtypes: int64(1), object(9)
memory usage: 9.3+ MB


It could find there are missing value in **ContractType**, **ContractTime** and **Company**

**Title**

In [33]:
title_counts = First_Train['Title'].value_counts()
print (title_counts)

Business Development Manager                                                  921
Project Manager                                                               749
Management Accountant                                                         661
Sales Executive                                                               603
Account Manager                                                               540
Mechanical Design Engineer                                                    459
Finance Manager                                                               458
Credit Controller                                                             439
Cleaner                                                                       439
Accounts Assistant                                                            435
Administrator                                                                 428
Recruitment Consultant                                                        423
Assistant Manage

In [34]:
print(len(title_counts))

135435
